In [ ]:
!pip install midi2audio
!pip install fluidsynth
!pip install --upgrade music21==6.7.1

!pip install gTTS 
!pip install pygobject
!pip install numpy
!pip install matplotlib
!pip install librosa
!pip install scipy
!pip install IPython
!pip install playsound 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4513 sha256=a4fb74e87c5889b637172226481bfea3852e59c1b4880898c90e0a17f7ca7dc9
  Stored in directory: /root/.cache/pip/wheels/44/4b/81/dee2d535b2dd27af13c230ba49aa6f05a5dff0f57c68acef55
Successfully built fluidsynth
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.2 MB 1.1 MB/s 
  Created wheel for music21: filename=music21-6.7.1-py3-none-any.whl size=21941719 sha256=ab7150f9546e2e9102bf7bff5019b30064e063b416218f657630de803d887cfd
  Stored in directory: /root/.cache/pip/wheels/72/44/61/90e4e65262ca1b4d9f707527b540729ce3f64e00fc6b38d54c
Successfully built music21
  Attempting uninstall: music21
    Found existing installation:

In [ ]:
# enables music21 to render images of musical notes
print('installing lilypond...')
!apt-get install lilypond > /dev/null

# converts midi files to wav files into order to play them
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

print('done!')

installing lilypond...
Extracting templates from packages: 100%
installing fluidsynth...
done!


In [ ]:
import music21
from music21 import *
from IPython.display import Image, Audio
import IPython
import pandas as pd

In [ ]:
import gtts
%matplotlib inline
import seaborn
import numpy, scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
from playsound import playsound
import time

#### Read the .mxl file

In [ ]:
c = music21.converter.parse('Sonata_in_G_3rd_mvt.mxl')

note_list = []
duration_list = []

for n in c.flat.notesAndRests:
  try:
    
    temp_name = n.pitch.name
    if n.duration.quarterLength == float(0.0):
      temp_name = str('Ornaments ') + str(n.pitch.name)

    note_list.append(str(temp_name) + str(n.pitch.octave))
    duration_list.append(n.duration.quarterLength)
  except:
    try:
      for i in n:
        None

      temp_note = []
      temp_duration = []

      for i in n:

        temp_note.append(str(i.pitch.name) + str(i.pitch.octave))
        temp_duration.append(i.duration.quarterLength)

      y = ''
      for i in temp_note:
        y += str(i)
        y = y + ','

      # Added
      x = ''
      for i in temp_duration:
        x += str(i)
        x = x + ','
      
    
      note_list.append(str('chord, ') + y + str('chord end'))
      duration_list.append(str(x[:-1]))

    except:

      try:
        note_list.append('Rest')
        duration_list.append(n.duration.quarterLength)
      except:
        raise 'It is not a note, chord or rest'

all_list = zip(note_list, duration_list)

#### Make it into data frame

In [ ]:
pd.set_option('display.max_rows', 1500)
df = pd.DataFrame()
df['Note type'] = note_list
df.insert(1, 'duration (in s)', duration_list)

#### Convert duration to the name of notes' type

In [ ]:
duration_dictionary = {float(0.0625):'note 64th',float(0.125):'note 32nd',float(0.25):'note 16th', float(0.5):'note 8th',float(1.0):'Quarter',float(2.0):'Half',float(4.0):'Whole',
                       float(0.0):'Ornament', float(1.5):'Quarter with augmentation Dot',  float(2.5):'Half with augmentation Dot',
                        float(0.75):'note 16th with augmentation Dot'} # Need to add more cases
duration_list_converted = []

for i in range(len(duration_list)):

  try:
    k = []
    temp = duration_dictionary[duration_list[i]]
    duration_list_converted.append(temp)

  except:
    k = []
    temp_2 = []
    k.append(duration_list[i].split(','))

    for j in k:
      for h in j:
        h = float(h)
        temp_2.append(duration_dictionary[float(h)])

    x = ''
    for j in temp_2:
      x = x + str(j) + ','
    duration_list_converted.append(x[:-1])


In [ ]:
df.insert(2, 'Note duration name', duration_list_converted)

#### Finding the key, time signature and metrome mark from .mxl file

In [ ]:
key = c.analyze('key')
meta_key = str(key)

dict_meta = dict()
dict_meta['key'] = meta_key

TimeSignature = c.recurse().getTimeSignatures()[0]
meta_TimeSignature = str(TimeSignature)
dict_meta['TimeSignature'] = meta_TimeSignature.split(' ')[-1][:-1].split('/')[0] + ' ' + meta_TimeSignature.split(' ')[-1][:-1].split('/')[-1]

metronomeMark = c.metronomeMarkBoundaries()[0][2]
metronomeMark = str(metronomeMark)
dict_meta['metronomeMark'] = metronomeMark.split(' ')[-1][:-1]

tempo = dict_meta['metronomeMark']
Time_Signature = dict_meta['TimeSignature']
keys = dict_meta['key']

#### Convert it into audio and save as .wav file/ .mp3 file


In [ ]:
a = note_list
b = duration_list_converted

plt.rcParams['figure.figsize'] = (13, 5)

# conbine two lists in alternation
result = [None]*(len(a)+len(b))
result[::2] = a
result[1::2] = b

output = ''

output = 'tempo, ' + tempo + ', time signature, ' + Time_Signature + ', keys, ' + keys + ', '

# list = 1159*2 = [:2318]
for i in result[2290:]:
  output += str(i)
  output = output + '... '

t1 = gtts.gTTS(output) 
# save the audio file  
t1.save('test.wav') 

#### speak the audio 

In [ ]:
x, sr = librosa.load('test.wav')
ipd.Audio('test.wav')

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
